# Tony's Recon Process
This script will do a full audit on all of Tony's receivers. The following aspects will be checked:
1. did all the files in the `raw_files` archive dir enter the `processed` dir?
2. did all the receivers from the files in the `raw_files` make it to the export location in the DB?
3. are any of the dangling images matching with ANY \[unfiltered] receiver in the export location?
4. are any of the dangling images matching with any receiver in the raw files?
This script will not auto resolve the issues, this will provide a status report

### Setup

In [1]:
from datetime import datetime
import json
import os
import re
import warnings
from pandas.errors import SettingWithCopyWarning

from stat import S_ISDIR

import pandas as pd
from IPython.core.display import HTML

from db import engine
from sftp_connection import get_sftp


FROM_DATE = '2024-01-01'
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

In [2]:
# Load SBT Vendors Table
sbt_vendors = pd.read_csv('../sbt_vendors.csv')

## Audit received and processed files
### Helpers to get files from SFTP, and others

In [3]:
# Helper to get list of valid files from sftp location
def get_sftp_files(full_directory_path: str, from_date: str = None) -> list:
    """
    Helper to get a list of files in a requested directory, optionally from a date
    Args:
        full_directory_path: full directory path for the files
        from_date: (optional) the oldest date (inclusive) from when to filter the files for, format: YYY-MM-DD

    Returns:
        a list of file names in the directory
    """
    from_date_timestamp = datetime.strptime(from_date, '%Y-%m-%d').timestamp() if from_date else None
    with get_sftp() as sftp:
        sftp.chdir(full_directory_path)
        files_list = [file for file in sftp.listdir_attr() if not S_ISDIR(file.st_mode)]

        if from_date_timestamp:
            files_list = [file for file in files_list if file.st_mtime >= from_date_timestamp]
        else:
            files_list = [file for file in files_list]

    return files_list

# Helper to read the files from sftp location
def read_sftp_files(full_directory_path: str, filenames_list: list) -> dict:
    """
    Helper to get the contents of the requested files from a requested directory
    Args:
        full_directory_path: full directory path for the files
        filenames_list: list of str file names to be read

    Returns:
        a dict map of filenames to file contents
    """
    filename_to_receivers_map = {}
    with get_sftp() as sftp:
        sftp.chdir(full_directory_path)
        for i, filename in enumerate(filenames_list):
            print(f'Reading file {i+1} of {len(filenames_list)}')
            with sftp.open(filename) as fp:
                filename_to_receivers_map[filename] = json.loads(fp.read())

    return filename_to_receivers_map

In [4]:
# Helper to normalise and count number of files
def get_dates_to_files_map(files: list) -> pd.DataFrame:
    """
    A Helper to get the count of files by date
    Args:
        files: list of files

    Returns:
        a date distribution of file count
    """
    filename_to_date_map = {'filename': [], 'date': []}
    for file in files:
        cleaned_filename = get_original_filename(file.filename)
        if cleaned_filename not in filename_to_date_map['filename']:
            filename_to_date_map['filename'].append(cleaned_filename)
            date = datetime.fromtimestamp(file.st_mtime).strftime('%Y-%m-%d')
            filename_to_date_map['date'].append(date)

    return pd.DataFrame(filename_to_date_map)

In [5]:
# clean the file names to get the original file name
def get_original_filename(filename: str) -> str:
    """
    Helper method to retrieve the original name of the uploaded file
        Uploaded files are renamed when moved to the raw files and processed files folders. The renaming for a file e.g. "receiver_uploads.json" is:
            raw_files/receiver_uploads__RANDINT.json
            processed/receiver_uploads__BATCHINT__INGSTID.json
    Args:
        filename: the name of the file post-processing

    Returns:
        str filename of the original file
    """
    name, ext = filename.split('.')
    original_name = name.split('__')[0]

    return f"{original_name}.{ext}"

In [6]:
# Set up FTP directory and get files list
root_directory = 'Tonys_Stores_Inbound'
from_date = FROM_DATE

# Get files in the raw files directory
raw_files_folder = 'raw_files'
raw_files_list = get_sftp_files(os.path.join(root_directory, raw_files_folder), from_date)

# Get files in processed folder
processed_folder = 'processed'
processed_files_list = get_sftp_files(os.path.join(root_directory, processed_folder), from_date)

In [7]:
# compare number of files from each folder for the given date range
raw_files_data = get_dates_to_files_map(raw_files_list)
processed_files_data = get_dates_to_files_map(processed_files_list)

In [8]:
# Do some cleanup on the raw files and the processed files to compare the counts
raw_files_count = raw_files_data.groupby(['date']).count().reset_index()
processed_files_count = processed_files_data.groupby(['date']).count().reset_index()
raw_to_processed_files_comparison_df = raw_files_count.merge(processed_files_count, on='date', suffixes=('_raw', '_processed'))
raw_to_processed_files_comparison_df['mismatch'] = raw_to_processed_files_comparison_df.apply(lambda x: x.filename_raw - x.filename_processed, axis=1)

In [9]:
# Track the files that may have not ingested
raw_files_names_list = raw_files_data.filename.unique().tolist()
processed_files_names_list = processed_files_data.filename.unique().tolist()
missing_in_processed_list = [filename for filename in raw_files_names_list if filename not in processed_files_names_list]

## Audit received and processed receivers

In [10]:
# Helpers to get exact match keys and fuzzy match keys
def get_match_key(row: dict, exact_match: bool = True) -> str:
    """
    Helper to get match key for exact matches
    Args:
        row:
        exact_match:

    Returns:
        tuple with the match key
    """
    pattern = re.compile("[\W_]+") if exact_match else re.compile("[^\d]")
    return f"{row['location_id']}__{row['vendor_id']}__{pattern.sub('', row['invoice_number'])}"


In [11]:
def receivers_json_to_df(filename_to_contents_dict: dict) -> pd.DataFrame:
    """
    Helper to convert dict of receivers to
    Args:
        filename_to_contents_dict:

    Returns:

    """
    receivers_map = {'invoice_number': [], 'location_id': [], 'vendor_id': [], 'total_amount': [], 'date': [], 'filename': []}
    for i, (filename, receivers_json_list) in enumerate(filename_to_contents_dict.items()):
        print(f'Reading file {i+1} of {len(filename_to_contents_dict)}')
        for j, receiver in enumerate(receivers_json_list):
            print(f'Parsing receiver {j+1} of {len(receivers_json_list)}')
            receivers_map['invoice_number'].append(receiver.get('InvoiceNumber'))
            receivers_map['location_id'].append(receiver.get('Location'))
            receivers_map['vendor_id'].append(receiver.get('Vendor'))
            receivers_map['total_amount'].append(receiver.get('Total'))
            receivers_map['date'].append(receiver.get('InvoiceDate'))
            receivers_map['filename'].append(filename)

    receivers_df = pd.DataFrame(receivers_map)
    receivers_df.drop_duplicates(inplace=True, keep='last')
    receivers_df['date'] = pd.to_datetime(receivers_df['date'])

    return receivers_df

In [12]:
# Get all receivers from the raw files
raw_files_to_receivers_json = read_sftp_files(os.path.join(root_directory, raw_files_folder), filenames_list=[file.filename for file in raw_files_list])
raw_receivers_df = receivers_json_to_df(raw_files_to_receivers_json)
raw_receivers_df.drop_duplicates(subset=['invoice_number', 'date', 'total_amount', 'vendor_id', 'location_id'], keep='first', inplace=True)
# Get all receivers from the processed files
processed_files_to_receivers_json = read_sftp_files(os.path.join(root_directory, processed_folder), filenames_list=[file.filename for file in processed_files_list])
processed_receivers_df = receivers_json_to_df(processed_files_to_receivers_json)
processed_receivers_df.drop_duplicates(subset=['invoice_number', 'date', 'total_amount', 'vendor_id', 'location_id'], keep='first', inplace=True)

Reading file 1 of 636
Reading file 2 of 636
Reading file 3 of 636
Reading file 4 of 636
Reading file 5 of 636
Reading file 6 of 636
Reading file 7 of 636
Reading file 8 of 636
Reading file 9 of 636
Reading file 10 of 636
Reading file 11 of 636
Reading file 12 of 636
Reading file 13 of 636
Reading file 14 of 636
Reading file 15 of 636
Reading file 16 of 636
Reading file 17 of 636
Reading file 18 of 636
Reading file 19 of 636
Reading file 20 of 636
Reading file 21 of 636
Reading file 22 of 636
Reading file 23 of 636
Reading file 24 of 636
Reading file 25 of 636
Reading file 26 of 636
Reading file 27 of 636
Reading file 28 of 636
Reading file 29 of 636
Reading file 30 of 636
Reading file 31 of 636
Reading file 32 of 636
Reading file 33 of 636
Reading file 34 of 636
Reading file 35 of 636
Reading file 36 of 636
Reading file 37 of 636
Reading file 38 of 636
Reading file 39 of 636
Reading file 40 of 636
Reading file 41 of 636
Reading file 42 of 636
Reading file 43 of 636
Reading file 44 of 6

In [13]:
# Match with exact matching key [This should line up perfectly because we simply copy the same file from one folder to the next; they should be identical]
raw_receivers_df['match_key'] = raw_receivers_df.apply(lambda x: get_match_key(x, exact_match=True), axis=1)
processed_receivers_df['match_key'] = processed_receivers_df.apply(lambda x: get_match_key(x, exact_match=True), axis=1)
raw_files_to_processed_files_missing_keys = [key for key in raw_receivers_df.match_key.tolist() if key not in processed_receivers_df.match_key.tolist()]

## Audit received receivers and receivers in DB

In [14]:
# Get receivers from DB. This would be all the invoices from Tony's export location (group 56351)
receivers_query = f"""
SELECT ii.invoice_number as invoice_number, ii.date as date, ii.total_amount as total_amount, split_part(ii.batch_number, '|', 1) as filename,
    rvad.vendor_name as vendor_id, ra.company_code as location_id
FROM invoices_invoice ii
JOIN restaurant_restaurant rr on ii.restaurant_id = rr.id
JOIN accounting_restaurantvendoraccountdetails rvad on rvad.id = ii.rvad_id
JOIN accounting_restaurantaccount ra on ra.restaurant_id = ii.restaurant_id
WHERE rr.group_id = 56351
    AND ii.date >= '{FROM_DATE}'
;
"""

db_receivers_df = pd.read_sql(receivers_query, engine)
db_receivers_df.drop_duplicates(subset=['invoice_number', 'date', 'total_amount', 'vendor_id', 'location_id'], keep='first', inplace=True)
engine.dispose()

In [15]:
# Do the same receiver parity checks but between raw and db receivers
raw_receivers_df['match_key'] = raw_receivers_df.apply(lambda x: get_match_key(x, exact_match=True), axis=1)
db_receivers_df['match_key'] = db_receivers_df.apply(lambda x: get_match_key(x, exact_match=True), axis=1)
# display(raw_receivers_df.head())
# display(db_receivers_df.head())

raw_to_db_missing_keys = [key for key in raw_receivers_df.match_key.tolist() if key not in db_receivers_df.match_key.tolist()]

# Check Dangling Images for matching Receivers

In [16]:
# Dangling images query
danling_images_query = """
SELECT ii.id, ii.invoice_number, ii.total_amount, ii.is_flagged, ii.date AS date,
    rvad.vendor_name AS vendor_id, ra.company_code AS location_id
FROM invoices_invoice  ii
JOIN restaurant_restaurant rr ON ii.restaurant_id = rr.id
JOIN accounting_restaurantvendoraccountdetails rvad ON rvad.id = ii.rvad_id
JOIN accounting_restaurantaccount ra ON ra.restaurant_id = ii.restaurant_id
WHERE 1=1
    AND rr.group_id = 56350
    AND ii.state IN (2, -2)
    AND ii.approved_date IS NULL
    AND ii.exported_date IS NULL
    AND ii.invoice_number IS NOT NULL;
"""

dangling_images_df = pd.read_sql(danling_images_query, engine)
engine.dispose()

In [17]:
# Do exact matches for dangling receivers from raw receivers files
left = dangling_images_df.copy()
left['date'] = pd.to_datetime(left['date'])

right = raw_receivers_df.copy()

left['match_key'] = left.apply(lambda x: get_match_key(x, exact_match=True), axis=1)
right['match_key'] = right.apply(lambda x: get_match_key(x, exact_match=True), axis=1)

exact_image_matches = right.merge(left, on='match_key', suffixes=('_image', '_right'))

if not exact_image_matches.empty:
    print(f'Found these matches:')
    display(exact_image_matches)

In [18]:
# Do fuzzy matches for dangling receivers from raw receivers files

remaining_images = left[~left.match_key.isin(exact_image_matches.match_key.tolist())]

if not remaining_images.empty:
    print(f'Trying to find fuzzy matches for remaining {len(remaining_images)} images:')

    left['match_key'] = left.apply(lambda x: get_match_key(x, exact_match=False), axis=1)
    right['match_key'] = right.apply(lambda x: get_match_key(x, exact_match=False), axis=1)

    fuzzy_image_matches = right.merge(left, on='match_key', suffixes=('_image', '_right'))

    if not fuzzy_image_matches.empty:
        print('Found these fuzzy matches:')
        display(fuzzy_image_matches)

    remaining_images = remaining_images[~remaining_images.match_key.isin(fuzzy_image_matches.match_key.tolist())]

    if not remaining_images.empty:
        print('Trying to do some more matches using vendor, amount, location, and date')
        alternate_attempt_df = remaining_images.merge(left, on=['vendor_id', 'location_id', 'total_amount', 'date'], suffixes=('_image', '_raw'))

        if not alternate_attempt_df.empty:
            print('Found these matches using vendor, amount, location, and date')
            display(alternate_attempt_df)

        remaining_images = remaining_images[~remaining_images.match_key.isin(alternate_attempt_df.match_key_image.tolist())]

        if not remaining_images.empty:
            print('Trying to do some more matches using amount, location, and date')
            final_attempt_df = remaining_images.merge(left, on=['location_id', 'total_amount', 'date'], suffixes=('_image', '_raw'))

            if not final_attempt_df.empty:
                print('Found these matches using amount, location, and date')
                display(final_attempt_df)

            remaining_images = remaining_images[~remaining_images.match_key.isin(final_attempt_df.match_key_image.tolist())]

            if remaining_images.empty:
                print('No more images left to check for')
            else:
                print('Could not match these remaining images:')
                display(remaining_images)
        else:
            print('No more images remaining to match')

    else:
        print('None Remaining')

Trying to find fuzzy matches for remaining 4 images:
Trying to do some more matches using vendor, amount, location, and date
Found these matches using vendor, amount, location, and date


,id_image,invoice_number_image,total_amount,is_flagged_image,date,vendor_id,location_id,match_key_image,id_raw,invoice_number_raw,is_flagged_raw,match_key_raw
0,88507856,52249490004840,149.40,False,2024-04-26,BIMDSD,2012,2012__BIMDSD__52249490004840,88507856,52249490004840,False,2012__BIMDSD__52249490004840
1,88515295,CM1848845,-192.00,False,2024-04-26,DUTCH06,2011,2011__DUTCH06__CM1848845,88515295,CM1848845,False,2011__DUTCH06__1848845
2,88538600,584640,54.00,False,2024-04-28,TORTI00,2002,2002__TORTI00__584640,88538600,584640,False,2002__TORTI00__584640
3,88516810,1787582,2646.15,False,2024-04-24,AMIGO06,2002,2002__AMIGO06__1787582,88516810,1787582,False,2002__AMIGO06__1787582


No more images remaining to match


## Reporting
### No. of Received & Processed Files

In [19]:
raw_files_data[raw_files_data.date=='2024-04-18']

,filename,date
204,02006_20240418150007.json,2024-04-18
252,02009_20240418150008.json,2024-04-18
302,02011_20240418150008.json,2024-04-18
351,02012_20240418150009.json,2024-04-18
419,02014_20240418150009.json,2024-04-18
607,DSD_20240417170504_ap001724190000.json,2024-04-18
611,DSD_20240418173504_ap301824193001.json,2024-04-18


In [20]:
# display(raw_to_processed_files_comparison_df)
print('Raw files to processed files comparison by date:')
display(raw_to_processed_files_comparison_df)


if missing_in_processed_list:
    print("Files in raw missing in processed folder:")
    display(raw_files_data[raw_files_data.filename.isin(missing_in_processed_list)])

    print("Receivers in the missed file(s):")
    raw_receivers_df['cleaned_name'] = raw_receivers_df.apply(lambda x: get_original_filename(x.filename), axis=1)
    display(raw_receivers_df[raw_receivers_df.cleaned_name.isin(missing_in_processed_list)])

else:
    display(HTML('<h3>All files in raw_files folder were found in processed folder</h3>'))


Raw files to processed files comparison by date:


,date,filename_raw,filename_processed,mismatch
0,2024-02-28,3,3,0
1,2024-02-29,15,15,0
2,2024-03-01,16,16,0
3,2024-03-02,13,13,0
4,2024-03-03,4,4,0
5,2024-03-04,14,14,0
6,2024-03-05,15,15,0
7,2024-03-06,16,16,0
8,2024-03-07,15,15,0
9,2024-03-08,15,15,0


In [21]:
# Receivers in raw files VS receivers in processed folder
if raw_files_to_processed_files_missing_keys:
    print('Following receivers were not found in processed folder:')
    raw_to_processed_missing_df = raw_receivers_df[raw_receivers_df.match_key.isin(raw_files_to_processed_files_missing_keys)]
    display(raw_to_processed_missing_df)

    print('Checking partial match in case processed folder invoice numbers have been altered...')
    raw_to_processed_missing_df['match_key'] = raw_to_processed_missing_df.apply(lambda x: get_match_key(row=x, exact_match=False), axis=1)
    processed_receivers_df['match_key'] = processed_receivers_df.apply(lambda x: get_match_key(row=x, exact_match=False), axis=1)

    # Check if any missing now
    fuzzy_missing_raw_keys = [key for key in raw_to_processed_missing_df.match_key.tolist() if key not in processed_receivers_df.match_key.tolist()]

    if fuzzy_missing_raw_keys:
        print('Following receivers are missing in processed even after fuzzy checks:')
        display(raw_receivers_df[raw_receivers_df.match_key.isin(fuzzy_missing_raw_keys)])

    else:
        print('All receivers in raw files were found in processed after trying fuzzy matching')

else:
    print('All receivers in raw files were found in processed folder files')

All receivers in raw files were found in processed folder files


### No. of receivers in raw_files folder and Export Location DB

In [22]:
# match the keys in raw files receivers list with that from the DB

if raw_to_db_missing_keys:
    print('Following receivers were not found in the DB using exact match:')
    raw_to_db_missing_df = raw_receivers_df[raw_receivers_df.match_key.isin(raw_to_db_missing_keys)]
    display(raw_to_db_missing_df)

    print('Trying to match with fuzzy invoice number matching...')
    raw_to_db_missing_df['match_key'] = raw_to_db_missing_df.apply(lambda x: get_match_key(row=x, exact_match=False), axis=1)
    db_receivers_df['match_key'] = db_receivers_df.apply(lambda x: get_match_key(row=x, exact_match=False), axis=1)

    # Check if any missing now
    fuzzy_missing_raw_keys = [key for key in raw_to_db_missing_df.match_key.tolist() if key not in db_receivers_df.match_key.tolist()]
    if fuzzy_missing_raw_keys:
        # This means we couldn't match with the exact check and fuzzy checks. We can only try a couple of different combo-checks to see if anything closely matches
        print('Following receivers were not found in DB even after fuzzy invoice name matching:')
        display(raw_to_db_missing_df[raw_to_db_missing_df.match_key.isin(fuzzy_missing_raw_keys)])

        # Now, do an alternate matching on amount, vendor, date, location to see how much diff is there in the invoice number
        left = raw_receivers_df[raw_receivers_df.match_key.isin(fuzzy_missing_raw_keys)][['vendor_id', 'location_id', 'total_amount', 'date', 'invoice_number', 'match_key']]
        right = db_receivers_df[['vendor_id', 'location_id', 'total_amount', 'date', 'invoice_number', 'match_key']]
        right['date'] = pd.to_datetime(right.date, errors='coerce')
        merged = left.merge(right, on=['vendor_id', 'location_id', 'total_amount', 'date'], suffixes=('_raw', '_db'))

        merged['same_inv_num?'] = merged.invoice_number_db == merged.invoice_number_raw
        if not merged.empty:
            print('Found some following possible matches:')
            display(merged)

        # Check if any more unmatched
        not_in_merged = left[~left.match_key.isin(merged.match_key_raw)]
        if not not_in_merged.empty:
            print('The following receivers were still not found:')
            display(not_in_merged)

            print('Trying one more possible matching using only dates, amounts, and locations...')
            another_merge = not_in_merged.merge(right, on=['total_amount', 'date', 'location_id'], suffixes=('_raw', '_db'))
            another_merge['same_inv_no?'] = another_merge.invoice_number_raw == another_merge.invoice_number_db

            if not another_merge.empty:
                print(f'Following receivers seem like they found a matching invoice:')
                display(another_merge)
            else:
                print('That did not match any receivers')

            still_missing = not_in_merged[~not_in_merged.match_key.isin(another_merge.match_key_raw)]

            if not still_missing.empty:
                print(f'The following receivers did not find a match anywhere in the DB')
                display(still_missing)

            else:
                print('No more missing receivers to check for')
        else:
            print('No more missing receivers to check for')
    else:
        print(f'All receivers in raw were found in DB after trying fuzzy invoice name matching. Trying another match using only amounts, vendors, dates, and locations.')


    # print(fuzzy_missing_raw_keys)
else:
    print('All receivers in raw were found in the DB')

Following receivers were not found in the DB using exact match:


,invoice_number,location_id,vendor_id,total_amount,date,filename,match_key
58,NS5364701,2001,DUTCH06,29.94,2024-03-12,02001_20240312150008__11068.json,2001__DUTCH06__NS5364701
66,9155,2001,CUGIN09,230.14,2024-03-13,02001_20240313150008__82977.json,2001__CUGIN09__9155
227,16190,2001,COUNT00,702.37,2024-04-17,02001_20240417150006__32673.json,2001__COUNT00__16190
265,3489,2001,MBFOO04,187.14,2024-04-24,02001_20240424150007__28115.json,2001__MBFOO04__3489
351,4740592,2002,JRDAI00,221.58,2024-03-13,02002_20240313150008__88558.json,2002__JRDAI00__4740592
...,...,...,...,...,...,...,...
40362,08224201325,2001,HERIT01,478.48,2024-04-26,DSD_20240426170505_ap002624190000__92916.json,2001__HERIT01__08224201325
40401,16802216,2012,TURAN00,30.36,2024-04-26,DSD_20240426170505_ap002624190000__92916.json,2012__TURAN00__16802216
40402,16802217,2012,TURAN00,58.10,2024-04-26,DSD_20240426170505_ap002624190000__92916.json,2012__TURAN00__16802217
40490,4050469,2001,CAMER15,649.12,2024-04-26,DSD_20240426170505_ap002624190000__92916.json,2001__CAMER15__4050469


Trying to match with fuzzy invoice number matching...
Following receivers were not found in DB even after fuzzy invoice name matching:


,invoice_number,location_id,vendor_id,total_amount,date,filename,match_key
66,9155,2001,CUGIN09,230.14,2024-03-13,02001_20240313150008__82977.json,2001__CUGIN09__9155
227,16190,2001,COUNT00,702.37,2024-04-17,02001_20240417150006__32673.json,2001__COUNT00__16190
265,3489,2001,MBFOO04,187.14,2024-04-24,02001_20240424150007__28115.json,2001__MBFOO04__3489
351,4740592,2002,JRDAI00,221.58,2024-03-13,02002_20240313150008__88558.json,2002__JRDAI00__4740592
370,07-071686,2002,LAGUA00,30.96,2024-03-19,02002_20240319150009__53830.json,2002__LAGUA00__07071686
...,...,...,...,...,...,...,...
40362,08224201325,2001,HERIT01,478.48,2024-04-26,DSD_20240426170505_ap002624190000__92916.json,2001__HERIT01__08224201325
40401,16802216,2012,TURAN00,30.36,2024-04-26,DSD_20240426170505_ap002624190000__92916.json,2012__TURAN00__16802216
40402,16802217,2012,TURAN00,58.10,2024-04-26,DSD_20240426170505_ap002624190000__92916.json,2012__TURAN00__16802217
40490,4050469,2001,CAMER15,649.12,2024-04-26,DSD_20240426170505_ap002624190000__92916.json,2001__CAMER15__4050469


Found some following possible matches:


...

,vendor_id,location_id,total_amount,date,invoice_number,match_key
227,COUNT00,2001,702.37,2024-04-17,16190,2001__COUNT00__16190
265,MBFOO04,2001,187.14,2024-04-24,3489,2001__MBFOO04__3489
1177,BARSFO,2006,528.47,2024-03-15,732407501,2006__BARSFO__732407501
1350,LIPAR09,2006,46.48,2024-04-16,1072316,2006__LIPAR09__1072316
3533,GOYAFO,2017,2686.00,2024-02-28,25289998,2017__GOYAFO__25289998
...,...,...,...,...,...,...
40288,GOYAFO,2011,271.59,2024-04-26,25450560,2011__GOYAFO__25450560
40289,GOYAFO,2011,56.22,2024-04-26,25450561,2011__GOYAFO__25450561
40290,GOYAFO,2002,806.81,2024-04-26,25451012,2002__GOYAFO__25451012
40291,GOYAFO,2002,104.38,2024-04-26,25451013,2002__GOYAFO__25451013


Trying one more possible matching using only dates, amounts, and locations...
Following receivers seem like they found a matching invoice:


,vendor_id_raw,location_id,total_amount,date,invoice_number_raw,match_key_raw,vendor_id_db,invoice_number_db,match_key_db,same_inv_no?
0,GOYAFO,2017,2686.00,2024-02-28,25289998,2017__GOYAFO__25289998,GOYAF00,25289998,2017__GOYAF00__25289998,True
1,GOYAFO,2008,3288.57,2024-02-28,25294505,2008__GOYAFO__25294505,GOYAF00,25294505,2008__GOYAF00__25294505,True
2,GOYAFO,2008,56.22,2024-02-28,25294506,2008__GOYAFO__25294506,GOYAF00,25294506,2008__GOYAF00__25294506,True
3,GOYAFO,2013,2129.85,2024-02-28,25294516,2013__GOYAFO__25294516,GOYAF00,25294516,2013__GOYAF00__25294516,True
4,GOYAFO,2006,1144.25,2024-02-29,25298431,2006__GOYAFO__25298431,GOYAF00,25298431,2006__GOYAF00__25298431,True
...,...,...,...,...,...,...,...,...,...,...
121,GOYAFO,2002,1270.76,2024-04-23,25439101,2002__GOYAFO__25439101,GOYAF00,25439101,2002__GOYAF00__25439101,True
122,GOYAFO,2009,318.00,2024-04-25,25446964,2009__GOYAFO__25446964,GOYAF00,25446964,2009__GOYAF00__25446964,True
123,GOYAFO,2011,271.59,2024-04-26,25450560,2011__GOYAFO__25450560,GOYAF00,25450560,2011__GOYAF00__25450560,True
124,GOYAFO,2002,104.38,2024-04-26,25451013,2002__GOYAFO__25451013,GOYAF00,25451013,2002__GOYAF00__25451013,True


The following receivers did not find a match anywhere in the DB


,vendor_id,location_id,total_amount,date,invoice_number,match_key
227,COUNT00,2001,702.37,2024-04-17,16190,2001__COUNT00__16190
265,MBFOO04,2001,187.14,2024-04-24,3489,2001__MBFOO04__3489
1177,BARSFO,2006,528.47,2024-03-15,732407501,2006__BARSFO__732407501
1350,LIPAR09,2006,46.48,2024-04-16,1072316,2006__LIPAR09__1072316
3537,GOYAFO,2013,50.70,2024-02-28,25294517,2013__GOYAFO__25294517
...,...,...,...,...,...,...
38765,GOYAFO,2001,924.06,2024-04-23,25439528,2001__GOYAFO__25439528
38766,GOYAFO,2011,1076.85,2024-04-23,25439716,2011__GOYAFO__25439716
40287,GOYAFO,2001,896.73,2024-04-26,25449187,2001__GOYAFO__25449187
40289,GOYAFO,2011,56.22,2024-04-26,25450561,2011__GOYAFO__25450561


### Check how many SBT Vendors

In [23]:
sbt_vendor_receivers_df = still_missing[still_missing.vendor_id.isin(sbt_vendors.VEND.tolist())]
if sbt_vendor_receivers_df.empty:
    print('No SBT vendor receivers')
else:
    print('Found the following SBT vendor receivers:')
    display(sbt_vendor_receivers_df)

No SBT vendor receivers


## Check for dangling invoice matches in the receivers